In [ ]:
# !Convert*solver_mosek.py*w*sh*

In [ ]:
import time
import math

import numpy

import mosek

In [ ]:
def mosek_set_model(mu, nu, c, task):
    m, n = c.shape
    
    inf = 0.
    
    task.appendvars(m*n)
    task.appendcons(m+n)
    
    task.putvarboundlist(
        range(m*n),
        [mosek.boundkey.lo]*(m*n),
        [0.]*(m*n),
        [inf]*(m*n)
    )
    
    for i in range(m):
        task.putarow(
            i,
            range(i*n, (i+1)*n),
            [1.]*n
        )
    task.putconboundlist(
        range(0, m),
        [mosek.boundkey.fx]*m,
        mu,
        mu
    )
    
    for i in range(n):
        task.putarow(
            i+m,
            range(i, i+m*n, n),
            [1.]*m
        )
    task.putconboundlist(
        range(m, m+n),
        [mosek.boundkey.fx]*n,
        nu,
        nu
    )
    
    task.putclist(range(m*n), c.reshape(m*n))
    
    task.putobjsense(mosek.objsense.minimize)

def solve_mosek(
    p,
    scale=None,
    mtd=None, sol=None, var=None, it=None,
    log=None, stat=False, title="",
    *args, **kwargs
):    
    if stat:
        start_time = time.time()
    
    m, n = p.c.shape
    
    if scale is None:
        scale = math.sqrt(m * n)
    
    with mosek.Env() as env:
        env.set_Stream(mosek.streamtype.log, log)
        
        with env.Task() as task:
            task.set_Stream(mosek.streamtype.log, log)
            
            task.putintparam(mosek.iparam.optimizer, mtd)
            
            mosek_set_model(scale*p.mu, scale*p.nu, p.c, task)
            
            if stat:
                end_time = time.time()
            
            task.optimize()
            
            xx = [0.] * (m*n)
            task.getxx(sol, xx)
            
            p.s = numpy.array(xx).reshape(m, n) / scale
    
            if stat:
                setup_time = end_time - start_time
                s = {
                    "title": title,
                    "size": [m, n],
                    "loss": task.getprimalobj(sol) / scale,
                    "vars": task.getintinf(var),
                    "iters": task.getintinf(it),
                    "setup": setup_time,
                    "solve": task.getdouinf(mosek.dinfitem.optimizer_time),
                }
                return p, s
            else:
                return p

In [ ]:
def solve_mosek_primal_simplex(p, *args, **kwargs):
    return solve_mosek(
        p,
        mtd=mosek.optimizertype.primal_simplex,
        sol=mosek.soltype.bas,
        var=mosek.iinfitem.opt_numvar,
        it=mosek.iinfitem.sim_primal_iter,
        title="MOSEK, simplex for primal",
        *args, **kwargs
    )

def solve_mosek_dual_simplex(p, *args, **kwargs):
    return solve_mosek(
        p,
        mtd=mosek.optimizertype.dual_simplex,
        sol=mosek.soltype.bas,
        var=mosek.iinfitem.opt_numvar,
        it=mosek.iinfitem.sim_dual_iter,
        title="MOSEK, simplex for dual",
        *args, **kwargs
    )

def solve_mosek_interior_point(p, *args, **kwargs):
    return solve_mosek(
        p,
        mtd=mosek.optimizertype.intpnt,
        sol=mosek.soltype.itr,
        var=mosek.iinfitem.opt_numvar,
        it=mosek.iinfitem.intpnt_iter,
        title="MOSEK, interior point",
        *args, **kwargs
    )

In [ ]:
# !ConvertEnd*

In [ ]:
# !Convert*solver_mosek_test.py*w*sehx*

In [ ]:
# !Switch*
from handler import FigureHandler
from dataset import ot_2d_Caffarelli
from stats import Statistics
# !SwitchCase*
# import font
# from handler import FigureHandler
# from dataset import ot_2d_Caffarelli
# from stats import Statistics
# from solver_mosek import solve_mosek_primal_simplex, solve_mosek_dual_simplex, solve_mosek_interior_point
# !SwitchEnd*

In [ ]:
# !Switch*
fh = FigureHandler(redir=True)
# !SwitchCase*
# fh = FigureHandler(sav=True, disp=False, ext=".pgf", redir=True)
# !SwitchEnd*

In [ ]:
stat = Statistics(
    probs=[
        ot_2d_Caffarelli(100, 100, 1)
    ],
    prob="Test problems",
    log=fh.write,
)

In [ ]:
stat.test(solve_mosek_primal_simplex, title="MOSEK, test", memory=True, prog=fh.write, clean=False)
stat.output_last()

In [ ]:
fh.fast(stat.probs[0].plot_link, aspect="equal")
stat.clean_last()

In [ ]:
stat.test(solve_mosek_primal_simplex, log=fh.write, prog=fh.write)
stat.output_last()

In [ ]:
stat.test(solve_mosek_dual_simplex, log=fh.write, prog=fh.write)
stat.output_last()

In [ ]:
stat.test(solve_mosek_interior_point, log=fh.write, prog=fh.write)
stat.output_last()

In [ ]:
# !ConvertEnd*